In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center> <img src="https://images.squarespace-cdn.com/content/v1/588f9607bebafbc786f8c5f8/1607924812500-Y1JR8L6XP5NKF2YPHDUX/image6.png?format=1000w"> </center>

* It is always considered as a good pratice to work on the copy of original dataset.

In [ ]:
main_df = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df = main_df.copy()
df.head()

* Importing the basic libraries required in this notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

Following are the list of algorithms that are used in this notebook.

| Algorithms      |
| ----------- |
| Logistic Regression      | 
| Decision Tree   | 
| Random Forest     | 
| XGBoost    | 
| KNeighbours      | 
| SVM   | 
| AdaBoost      | 

### EDA - Exploratory Data Analysis

In [ ]:
# Shape of dataframe

df.shape

In [ ]:
# List of all columns

df.columns

In [ ]:
# Basic Information about the dataframe

df.info()

In [ ]:
# List of all features with number of unique values present in them

df.nunique()

In [ ]:
# Statistical measure of dataset

df.describe()

In [ ]:
# We are dropping CustomerID because it will not make any contribution in prediction.

df = df.drop('customerID', axis=1)
df.head()

In [ ]:
# Here totalcharges are object types so we need to change it into numeric format
df['TotalCharges'].dtype

In [ ]:
# Changing in numeric format

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors="coerce")

In [ ]:
# Checking for null values 

df.isnull().sum()

In [ ]:
# Dropping all the rows in which value in not known

df.drop(df[df['TotalCharges'].isnull()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

## Visualization

In [ ]:
# Visualization of null values

sns.heatmap(df.isnull())

In [ ]:
# Checking Correlation 

df.corr()

In [ ]:
# Visualizing correalation

sns.heatmap(df.corr(), annot=True)

In [ ]:
plt.figure(figsize=(5, 5))
sns.barplot(data = df, y="TotalCharges", x="Churn")

In [ ]:
sns.barplot(data = df, x="tenure", y="Churn")

In [ ]:
px.scatter(df, y="TotalCharges", x="tenure")

In [ ]:
diag = px.histogram(df, x="Churn", color="SeniorCitizen")
diag.update_layout(width=750, height=550)
diag.show()

In [ ]:
diag = px.pie(df, values='TotalCharges', names='Churn', hole=0.5)
diag.show()

In [ ]:
labels = df['MultipleLines'].unique()
values = df['MultipleLines'].value_counts()

# pull is given as a fraction of the pie radius
diag = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0.1, 0.2])])
diag.show()

In [ ]:
labels = df['InternetService'].unique()
values = df['InternetService'].value_counts()


diag = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0.2, 0.3])])
diag.show()

In [ ]:
labels = df['PaymentMethod'].unique()
values = df['PaymentMethod'].value_counts()


diag = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
diag.show()

In [ ]:
labels = df['Contract'].unique()
values = df['Contract'].value_counts()


diag = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0.2, 0.3])])
diag.show()

### Preprocessing Data

In [ ]:
print (df['Partner'].value_counts(ascending=True))

In [ ]:
for i in df.columns:
    if df[i].dtypes=="object":
        print(f'{i} : {df[i].unique()}')
        print("****************************************************")

* Above cell shows us the list of all features with their respective categorical variables

In [ ]:
df.replace('No internet service', 'No', inplace=True)
df.replace('No phone service', 'No', inplace=True)

* Replace long negative text by "No" for ease of access

In [ ]:
for i in df.columns:
    if df[i].dtypes=="object":
        print(f'{i} : {df[i].unique()}')
        print("****************************************************")

* After Removing all long values we have above dataframe

In [ ]:
print(df['gender'].value_counts(ascending=True))

In [ ]:
# Replaceing Male be 0 and Female by 1

df['gender'].replace({'Female':1,'Male':0},inplace=True)
df.head()

In [ ]:
print(df['InternetService'].value_counts(ascending=True))

* There are few features in which categorical variables are more than two and they are not "Yes" or "No" types

In [ ]:
for i in df.columns:
    if (len(df[i].unique()) >2) & (df[i].dtypes != "int64") &(df[i].dtypes!= "float64"):
        print(i)

* Above 3 Features column in dataset have more than 2 categorical values

In [ ]:
print(df['InternetService'].value_counts(ascending=True))

In [ ]:
print(df['Contract'].value_counts(ascending=True))

In [ ]:
print(df['PaymentMethod'].value_counts(ascending=True))

In [ ]:
len(df['PaymentMethod'].unique())

* To deal with such kind of feature column we are required to use one-hot encoding.

In [ ]:
more_than_2 = ['InternetService' ,'Contract' ,'PaymentMethod']
df = pd.get_dummies(data=df, columns= more_than_2)
df.dtypes

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
for i in df.columns:
    if (df[i].dtypes == "int64")  | (df[i].dtypes== "float64"):
        print(i)

* Above feature column have numerical data so we will need to bring it into a particular range if they varies a lot

In [ ]:
# Using MinMaxScaler of Feature Scaling

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

* Here we will consider gender and seniorcitizen because because their values already present in a small scale of 0 or 1
* So we will consider only "tenure", "MonthlyCharges", "TotalCharges"

In [ ]:
large_cols = ["tenure", "MonthlyCharges", "TotalCharges"]
df[large_cols] = scaler.fit_transform(df[large_cols])
df[large_cols].head()

In [ ]:
# After feature scaling we have following dataset

df.head()

In [ ]:
for i in df.columns:
    if (df[i].dtypes == "object"):
        print(i)

* All the column present above have object type of dataset in them i.e here only "Yes" or "No"

In [ ]:
two_cate = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']
for i in two_cate:
    df[i].replace({"No":0, "Yes":1}, inplace=True)
df.head()

In [ ]:
# Splitting Dataset into train and test set

X = df.drop('Churn', axis=1)
y = df['Churn']

In [ ]:
X.shape, y.shape

In [ ]:
# Performing Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

## Model Building

### Using Logistics Regression

In [ ]:
# Importing Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# creating object for model
model_lg = LogisticRegression(max_iter=120,random_state=0, n_jobs=20)

In [ ]:
# Model Training

model_lg.fit(X_train, y_train)

In [ ]:
# Making Predictions
pred_lg = model_lg.predict(X_test)

In [ ]:
# Calculating Accuracy of the model

lg = round(accuracy_score(y_test, pred_lg)*100,2)
print(lg)

In [ ]:
# Classification Report

print(classification_report(y_test, pred_lg))

In [ ]:
# confusion Matrix

cm1 = confusion_matrix(y_test, pred_lg)
sns.heatmap(cm1/np.sum(cm1), annot=True, fmt='0.2%', cmap="Reds")
plt.title("Logistic Regression Confusion Matrix",fontsize=12)
plt.show()

### Using decision Tree Classifer

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Creating object of the model
model_dt = DecisionTreeClassifier(max_depth=4, random_state=42)

In [ ]:
model_dt.fit(X_train, y_train)

In [ ]:
pred_dt = model_dt.predict(X_test)

In [ ]:
dt  = round(accuracy_score(y_test, pred_dt)*100, 2)
print(dt)    

In [ ]:
print(classification_report(y_test, pred_dt))

In [ ]:
# confusion Maxtrix
cm2 = confusion_matrix(y_test, pred_dt)
sns.heatmap(cm2/np.sum(cm2), annot = True, fmt=  '0.2%', cmap = 'Reds')
plt.title("Decision Tree Classifier Confusion Matrix",fontsize=12)
plt.show()

### Using Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Creating model object
model_rf = RandomForestClassifier(n_estimators=300,min_samples_leaf=0.16, random_state=42)

In [ ]:
# Training Model
model_rf.fit(X_train, y_train)

In [ ]:
# Making Prediction
pred_rf = model_rf.predict(X_test)

In [ ]:
# Calculating Accuracy Score
rf = round(accuracy_score(y_test, pred_rf)*100, 2)
print(rf)

In [ ]:
print(classification_report(y_test,pred_rf))

In [ ]:
# confusion Maxtrix
cm3 = confusion_matrix(y_test, pred_rf)
sns.heatmap(cm3/np.sum(cm3), annot = True, fmt=  '0.2%', cmap = 'Reds')
plt.title("RandomForest Classifier Confusion Matrix",fontsize=12)
plt.show()

### Using XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# Creating model object
model_xgb = XGBClassifier(max_depth= 8, n_estimators= 125, random_state= 0,  learning_rate= 0.03, n_jobs=5)

In [ ]:
# Training Model
model_xgb.fit(X_train, y_train)

In [ ]:
# Making Prediction
pred_xgb = model_xgb.predict(X_test)

In [ ]:
# Calculating Accuracy Score
xgb = round(accuracy_score(y_test, pred_xgb)*100, 2)
print(xgb)

In [ ]:
print(classification_report(y_test,pred_xgb))

In [ ]:
# confusion Maxtrix
cm4 = confusion_matrix(y_test, pred_xgb)
sns.heatmap(cm4/np.sum(cm4), annot = True, fmt=  '0.2%', cmap = 'Reds')
plt.title("XGBoost Classifier Confusion Matrix",fontsize=12)
plt.show()

### Using KNeighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Creating model object
model_kn = KNeighborsClassifier(n_neighbors=9, leaf_size=20)

In [ ]:
# Training Model
model_kn.fit(X_train, y_train)

In [ ]:
# Making Prediction
pred_kn = model_kn.predict(X_test)

In [ ]:
# Calculating Accuracy Score
kn = round(accuracy_score(y_test, pred_kn)*100, 2)
print(kn)

In [ ]:
print(classification_report(y_test,pred_kn))

In [ ]:
# confusion Maxtrix
cm5 = confusion_matrix(y_test, pred_kn)
sns.heatmap(cm5/np.sum(cm5), annot = True, fmt=  '0.2%', cmap = 'Reds')
plt.title("KN Classifier Confusion Matrix",fontsize=12)
plt.show()

###  Using SVM

In [ ]:
from sklearn.svm import SVC, LinearSVC

In [ ]:
model_svm = SVC(kernel='rbf', random_state = 42)

In [ ]:
model_svm.fit(X_train, y_train)

In [ ]:
# Making Prediction
pred_svm = model_svm.predict(X_test)

In [ ]:
# Calculating Accuracy Score
sv = round(accuracy_score(y_test, pred_svm)*100, 2)
print(sv)

In [ ]:
print(classification_report(y_test,pred_kn))

In [ ]:
# confusion Maxtrix
cm6 = confusion_matrix(y_test, pred_svm)
sns.heatmap(cm6/np.sum(cm6), annot = True, fmt=  '0.2%', cmap = 'Reds')
plt.title("SVM Classifier Confusion Matrix",fontsize=12)
plt.show()

### Using AdaBoost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
model_ada = AdaBoostClassifier(learning_rate= 0.002,n_estimators= 205,random_state=42)

In [ ]:
model_ada.fit(X_train, y_train)

In [ ]:
# Making Prediction
pred_ada = model_ada.predict(X_test)

In [ ]:
# Calculating Accuracy Score
ada = round(accuracy_score(y_test, pred_ada)*100, 2)
print(ada)

In [ ]:
print(classification_report(y_test,pred_ada))

In [ ]:
# confusion Maxtrix
cm7 = confusion_matrix(y_test, pred_ada)
sns.heatmap(cm7/np.sum(cm7), annot = True, fmt=  '0.2%', cmap = 'Reds')
plt.title("Adaboost Classifier Confusion Matrix",fontsize=12)
plt.show()

In [ ]:
models = pd.DataFrame({
    'Model':['Logistic Regression', 'Decision Tree', 'Random Forest', 'XGBoost', 'KNeighbours', 'SVM', 'AdaBoost'],
    'Accuracy_score' :[lg, dt, rf, xgb, kn, sv, ada]
})
models
sns.barplot(x='Accuracy_score', y='Model', data=models)

models.sort_values(by='Accuracy_score', ascending=False)

### Conclusion :- Above graph shows the performance and comparsion of different models used in this notebook.